In [ ]:
!pip install torch==1.12.1

In [ ]:
!pip install timm==0.6.12

In [ ]:
import os
import sys
import numpy as np
import pandas as pd

from timm import create_model
import fastai
from fastai.vision.all import *

set_seed(999, reproducible=True)

In [ ]:
dataset_path_train = '../input/codenrockart/'

dataset_path_test = '../input/codenrockart/test/test'

In [ ]:
# model names in timm, corresponding image input sizes and weights in the final model blend

TIMM_MODELS = (('swinv2_large_window12to16_192to256_22kft1k', 384, 0.189), ('swin_large_patch4_window12_384', 384, 0.168), ('vit_large_patch14_clip_336', 336, 0.063), 
               ('beit_large_patch16_384', 384, 0.261), ('beitv2_large_patch16_224', 224, 0.087), ('beit_large_patch16_224.in22k_ft_in22k', 224, 0.232))

In [ ]:
train_df = pd.read_csv(dataset_path_train/'train.csv', sep='\t')
train_df['path'] = train_df['image_name'].map(lambda x: str(dataset_path_train/'train_data/train'/x))
train_df = train_df.drop(columns=['image_name'])
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [ ]:
file_list = [f for f in os.listdir(dataset_path_test)]

test_df = pd.DataFrame(file_list, columns=['image_name'])
test_df['path'] = test_df['image_name'].map(lambda x:str(dataset_path_test+'/'+x))
test_df['label_id'] = [0]*len(test_df)

In [ ]:
def train_and_predict(model):

    model_name = model[0]
    img_size = model[1]

    dls = ImageDataLoaders.from_df(train_df,
                               valid_pct=0.0,
                               seed=999,
                               fn_col='path',
                               label_col='label_id',
                               y_block=CategoryBlock,
                               bs=8,
                               num_workers=2,
                               item_tfms=Resize(img_size),
                               batch_tfms=aug_transforms(size=img_size))

    model = create_model(model_name, pretrained=True, num_classes=dls.c)

    learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)

    learn.fit_one_cycle(3, 2e-5)

    test_dl = dls.test_dl(test_df)

    prediction, _ = learn.tta(dl=test_dl, n=7)

    return prediction

In [ ]:
prediction_blend = []

for model in TIMM_MODELS:

    prediction = train_and_predict(model)

    model_weight = model[2]

    prediction_blend.append(model_weight*prediction)

In [ ]:
prediction_final = sum(prediction_blend)

test_df['label_id'] = np.argmax(prediction_final, axis=1)

test_df[['image_name', 'label_id']].to_csv('submission.csv', index=False, sep='\t')